In [1]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS
import time

In [2]:
key='d0882ea7c5df3b123a9450d91929525f23bfa3ad'
headers={'User-Agent':'AsherKetchums Video Game Analysis Project'}

def print_refresh(string):
    print(string,
          '\t\t\t\t\t\t\t\t\t\t\t\t',
          end='\r',
          flush = True
         )
    
def pause_update(n):
    for i in range(n):
        print_refresh(f'Waiting {n} seconds: {n-i} seconds remaining')
        time.sleep(1)
    print(f'Finished with {n} second wait')

In [4]:
games_df=pd.read_csv('../data/games_df.csv')

C:\Users\asher\AppData\Local\Temp\ipykernel_80248\3940341832.py:1: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df=pd.read_csv('../data/games_df.csv')


In [5]:
game_ids=games_df['game_id'].unique()

In [6]:
game_ids_list=games_df['game_id'].tolist()

In [7]:
game_ids=pd.DataFrame(game_ids)
game_ids_list=game_ids[0].tolist()

In [ ]:
#ALL-GAMES
game_url = 'https://www.giantbomb.com/api/game/'

games=[]

for ids in game_ids_list:
    #offset=0
    time.sleep(15)
    
    params={
    'api_key':key,
    'format':'json',
    'field_list':'characters,concepts,deck,developers,expected_release_day,expected_release_month,expected_release_year,expected_release_quarter,first_appearence_characters,first_appearence_concepts,first_appearence_locations,first_appearence_objects,franchises,genres,id,image,killed_characters,locations,number_of_user_reviews,platforms,publishers,themes,name,original,game_rating,original_release_date',
    #'limit':50, #Changed limit due to weight of queries per call on the server. I believe this is why 503 error was so common
    #'offset':offset
        }
    
    headers=headers
    try:
        response = requests.get(f'{game_url}{ids}/',params=params,headers=headers,timeout=30)
    except (requests.exceptions.ReadTimeout, requests.exceptions.ConnectionError) as re:
        print(f'Network Error:{re} damn')
        pause_update(600)
        continue

    if response.status_code == 204:
        print('2 0 not 4 sure if this is necessary')
        pause_update(600)
        continue

    if response.status_code == 420:
        print('420 and chill bruh')
        pause_update(600)
        continue

    if response.status_code == 503:
        print('server overload (503)')
        pause_update(60)
        continue
    
    response.raise_for_status()

    try:
        res=response.json()
    except ValueError:
        print('Value Error, jason')
        pause_update(600)
        continue
            
    results=res.get('results',None)
    if results:
        games.extend(results)
        
    time.sleep(15)